In [1]:
# Import Kubeflow SDK
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [ ]:
def load_data(data_path):
    
     # func_to_container_op requires packages to be imported inside of the function.
    import os
    import paramiko
    import os
    from os import walk
    import pandas as pd
    
    ###### download datasets from git #######
    dataset_url = "https://github.com/ZoieD/Hernia_X-ray_images_sample/raw/main/Hernia_sample10.tgz"
    data_dir = tf.keras.utils.get_file(origin=dataset_url,
                               fname='Hernia_sample',
                               untar=True)
    data_dir = pathlib.Path(data_dir)
    # 1 - Open a transport
    host="10.60.1.141"
    port = 22
    transport = paramiko.Transport((host, port))

    # 2 - Auth
    password="P@ssw0rd"
    username="user"
    transport.connect(username = username, password = password)

    # 3 - Go!
    sftp = paramiko.SSHClient()
    sftp._transport = transport
    sftp_download = sftp.open_sftp()
    # 4 - list all the files
    source_folder="datasets-registry/Hernia_sample10"
    cmd_line = 'find '+source_folder+' ' +'-type f'
    stdin, stdout, stderr = sftp.exec_command(cmd_line)
    test = stdout.read().decode("utf-8")
    test1 = test.splitlines()
    # 5 - Download the files and put in the local folder
    for file in test1:
        print(file)
        file_name = file.split('/')[-1]
        if file_name:
            base_path = file.split(file_name)[0]
            if not os.path.exists(base_path):
                os.makedirs(base_path, exist_ok=True)

            if file_name.split('.')[-1] in ('csv'):
                sftp_download.get(file, file)

    df = pd.read_csv('datasets-registry/Hernia_sample10/dataset.csv', sep=",", encoding='utf-8')        

    df.to_csv(f'{data_path}/dataset.csv', sep=",", index=False)
    print(df.head())

In [ ]:
def train(data_path, model_file):
    # func_to_container_op requires packages to be imported inside of the function.
    import os
    import math
    import warnings
    import pandas as pd
    import numpy as np
    from sklearn.utils import shuffle, resample
    from sklearn.metrics import  confusion_matrix, f1_score, roc_auc_score, roc_curve, auc, classification_report, accuracy_score, ConfusionMatrixDisplay
    from sklearn.metrics import cohen_kappa_score

    import tensorflow as tf
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import Sequential, load_model, Model
    from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization
    from tensorflow.keras.applications import VGG16, VGG19, ResNet101V2, ResNet50V2, InceptionV3, InceptionResNetV2, NASNetLarge, DenseNet121, DenseNet169, DenseNet201, Xception
    from tensorflow.keras.optimizers import Nadam, SGD, RMSprop, Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.metrics import binary_accuracy, categorical_crossentropy
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.backend import clear_session
    from sklearn.model_selection import train_test_split
    from sklearn.utils import shuffle, resample
    import sys
    import argparse
    import joblib
    import json
    
    #defining the install function
    import subprocess
    def install(name):
        subprocess.call(['pip', 'install', name])
    
    #install packages (installing numpy for the sake of demo)
    install('paramiko')
    import paramiko
    
    host="10.60.1.141"
    port = 22
    transport = paramiko.Transport((host, port))

    # 2 - Auth
    password="P@ssw0rd"
    username="user"
    transport.connect(username = username, password = password)

    # 3 - Go!
    sftp = paramiko.SSHClient()
    sftp._transport = transport
    sftp_download = sftp.open_sftp()
    # 4 - list all the files
    source_folder="datasets-registry/Hernia_sample10"
    cmd_line = 'find '+source_folder+' ' +'-type f'
    stdin, stdout, stderr = sftp.exec_command(cmd_line)
    test = stdout.read().decode("utf-8")
    test1 = test.splitlines()
    # 5 - Download the files and put in the local folder
    
    for file in test1:
        print(file)
        file_name = file.split('/')[-1]
        if file_name:
            base_path = file.split(file_name)[0]
            if not os.path.exists(base_path):
                os.makedirs(base_path, exist_ok=True)

            if file_name.split('.')[-1] in ('jpg', 'png'):
                sftp_download.get(file, file)
    
    df = pd.read_csv(f'{data_path}/dataset.csv', sep=",")
    print(df['labels'].value_counts())
    
    train_val_df, test_df = train_test_split(
        df,
        test_size = 0.2,
        random_state = 123,
        stratify=df['labels']
    )
    train_df, val_df = train_test_split(
        train_val_df,
        test_size = 0.2,
        random_state = 123,
        stratify=train_val_df['labels']
    )

    # when use v1
    # check whether need to do oversampling for positive datasets 
    pos_num = len(df[df["labels"] != '0'])
    neg_num = len(df[df["labels"] == '0'])
    print("positive:", pos_num, "negative:", neg_num)

    if pos_num+100 < neg_num:
        # do oversampling
        selected_df = pd.DataFrame(columns=['image_index', 'labels'])
        pos_sub_df = train_df[train_df["labels"] != '0']
        neg_sub_df = train_df[train_df["labels"] == '0']
        # print("negative size", len(neg_sub_df))
        pos_tem_df = resample(pos_sub_df,
                               replace=True,
                               n_samples=2500,
                               random_state=10)
        selected_df = selected_df.append(pos_tem_df[['image_index', 'labels']])
        
        
        neg_tem_df = resample(neg_sub_df,
                                   replace=True,
                                   n_samples=2500,
                                   random_state=10)
        selected_df = selected_df.append(neg_tem_df[['image_index', 'labels']])
        train_df = selected_df 
    
    labels = train_df['labels'].unique()
    labels = list(labels)
    
    train_datagen = ImageDataGenerator(rescale=1./255.,
                                    rotation_range=20, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    horizontal_flip=True, 
                                    vertical_flip=False,
                                    fill_mode="nearest")

    val_test_datagen = ImageDataGenerator(rescale=1./255.)

    train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                        #directory=IMG_PATH,
                                                        x_col='image_index',
                                                        y_col='labels',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='binary',
                                                        seed = 42,
                                                        shuffle=True,
                                                        # classes=['0', 'Hernia'],
                                                        classes = labels,
                                                        interpolation='nearest')

    val_generator = val_test_datagen.flow_from_dataframe(dataframe=val_df,
                                                         #directory=IMG_PATH,
                                                         x_col='image_index',
                                                         y_col='labels',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='binary',
                                                         seed = 42,
                                                         classes = labels,
                                                        #  classes=['0', 'Hernia'],
                                                         shuffle=True
                                                         )

    test_generator = val_test_datagen.flow_from_dataframe(dataframe=test_df,
                                                          #directory=IMG_PATH,
                                                          x_col='image_index',
                                                          y_col='labels',
                                                          target_size=(224, 224),
                                                          batch_size=1,
                                                          class_mode='binary',
                                                          classes = labels,
                                                        #   classes=['0', 'Hernia'],
                                                          shuffle=False)
    print("#####################################")
    print("indices", train_generator.class_indices)

    inceptresnet = InceptionResNetV2(
        weights='imagenet',
        input_shape=(224, 224, 3),
        include_top=False)

    x = inceptresnet.output
    x = GlobalAveragePooling2D(name="gap")(x)
    x = Dense(256, activation='elu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    pred = Dense(1, activation = "sigmoid", name="fc_out", kernel_initializer='he_uniform')(x)
    model = Model(inputs=inceptresnet.input, outputs=pred)

    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # model_file = "models/Hernia" -- supposed to pass in
#     try:
#         os.makedirs(model_file, exist_ok=True)
#         print("Directory '%s' created successfully" %model_file)
#     except OSError as error:
#         print("Directory '%s' can not be created")

#     model_saved_path = os.path.join(model_file, "model.h5")

    checkpoint = ModelCheckpoint(f'{data_path}/{model_file}', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')
                                
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

    callbacks_list = [checkpoint, reduce_lr, earlyStopping]

    model.fit(train_generator, 
            steps_per_epoch=math.ceil(train_generator.n/train_generator.batch_size),
            epochs=5,
            validation_data=val_generator,
            validation_steps=math.ceil(val_generator.n/val_generator.batch_size),
            callbacks=callbacks_list)
    
    ## predict
    if not model:
        model = load_model(f'{data_path}/{model_file}')
        test_generator.reset()
    y_pred = model.predict(test_generator, steps=(math.ceil(test_generator.n/test_generator.batch_size)), verbose=1)
    y_true = test_generator.classes

    #evaluation measures
    auc = roc_auc_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred >= 0.5)
    acc = accuracy_score(y_true, y_pred >= 0.5)
    #fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    #kappa is usually for imbalanced classes
    kappa_score = cohen_kappa_score(y_true, y_pred >= 0.5)
    
    cm = confusion_matrix(y_true, y_pred >= 0.5)    
    #TN, FP, FN, TP = confusion_matrix(y_true, y_pred >= 0.5).ravel()
    TN, FP, FN, TP = cm.ravel()
    print (TN, FP, FN, TP)
    
    #sensitivity or true positive rate
    sensitivity = TP/(TP+FN)
    #specificity or true negative rate
    specificity = TN/(TN+FP)
    #false positive rate
    FPR = FP/(FP+TN)
    #precision, positive predictive value
    PPV = TP/(TP+FP)
    #negative predictive value
    NPV = TN/(TN+FN)

    # print("InceptionResNetV2 model (weights=%f, img_w=%f, img_h=%f, channel=%f):" % (weights, img_w, img_h, channel))
    print ('AUC: ', round(auc, 3))
    print ('F1-score: ', round(f1, 3))
    print ('Sensitivity: ', round(sensitivity, 3))
    print ('Specificity: ', round(specificity, 3))
    print ('False positive rate:', round(FPR, 3))
    print ('PPV: ', round(PPV, 3))
    print ('NPV: ', round(NPV, 3))
    print ('Accuracy: ', round(acc, 3))
    print ('Kappa Score: ', round(kappa_score, 3))
    
    auc = auc.tolist()
    f1 = f1.tolist()
    acc = acc.tolist()
    cm = cm.tolist()

    with open(f'{data_path}/scores.json', "w") as f:
        scores = {
            "auc": auc,
            "f1": f1,
            "cm": cm,
            "acc": acc
            }
    
        json.dump(scores, f, indent=4)
    print("result saved successfully")

In [ ]:
# Create train and predict lightweight components, converting functions to container operation
load_data_op = comp.create_component_from_func(load_data, base_image='registry.hpe.com:5000/pipeline:load_data')
train_op = comp.create_component_from_func(train, base_image='registry.hpe.com:5000/pipeline:train_v1')

In [ ]:
# Define the pipeline
@dsl.pipeline(
   name='Lung Disease Detection Pipeline',
   description='Lung Disease Detection Pipeline to be executed on KubeFlow.'
)

# Define parameters to be fed into pipeline
def lung_disease_detection_container_pipeline(
    data_path: str, 
    model_file: str
):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWM)

    # Create lung disease detection:loading data component.
    load_data_container = load_data_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create lung disease detection:train and evaluate component.
    train_container = train_op(data_path, model_file) \
                                    .add_pvolumes({data_path: load_data_container.pvolume})

    # Make a kfp.dsl.ContainerOp in Python that defines how Kubeflow Pipelines interacts with our container, specifying:
            # The Docker image location to use
            # How to pass arguments to the running container
            # What outputs to expect from the container
    #Print the metrics of the model. 
    lung_disease_detection_result_container = dsl.ContainerOp(
        name="print_metrics", # What will show up on the pipeline viewer
        image='library/bash:4.4.23', # The image that KFP runs to do the work
        pvolumes={data_path: train_container.pvolume},# Dictionary to match a path on the op’s fs with a V1Volume or it inherited type.
        arguments=['cat', f'{data_path}/scores.json'] # the arguments of the command. 
        )

In [ ]:
DATA_PATH = '/mnt'
MODEL_FILE='hernia_model.h5'

In [ ]:
pipeline_func = lung_disease_detection_container_pipeline

In [ ]:
client = kfp.Client()

In [ ]:
# compile and run the pipeline
import time
experiment_name = 'lung_disease_detection_pipeline'
run_name = pipeline_func.__name__ + ' run'
namespace = 'zoie-workspace'

arguments = {"data_path":DATA_PATH,
             "model_file": MODEL_FILE}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function 
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name + '-' + time.strftime("%Y%m%d-%H%M%S"), 
                                                  run_name=run_name, 
                                                  arguments=arguments,
                                                  namespace=namespace)